In [14]:
import pandas as pd
import os
import glob
import re

In [15]:
data_dir = "output"
csv_files_name = "naver_stock_articles_*.csv"
numeric_columns = ['article_viewers', 'article_likes', 'article_dislikes']
cleaned_data_dir = 'cleaned_output'
df_list = []
before_data_row = 0
after_data_row = 0

if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    print(f"'{data_dir}' 폴더를 생성했습니다. 분석할 CSV 파일들을 이 폴더에 넣어주세요.")
    
if not os.path.exists(cleaned_data_dir):
    os.makedirs(cleaned_data_dir)
    print(f"'{cleaned_data_dir}' 폴더를 생성했습니다. 중복 제거 및 기본 전처리 데이터가 저장됩니다.")
    
def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = re.sub(r'[\r\n\t]+', ' ', text) # 줄바꿈, 탭을 공백으로
    text = text.strip() # 양 끝 공백 제거
    return text

In [16]:
# output 폴더에서 csv_files_name과 일치하는 모든 csv 파일을 찾아서 리스트 형태로 저장
csv_files = glob.glob(os.path.join(data_dir, csv_files_name))

if not csv_files:
    print(f"'{data_dir}' 폴더에 CSV 파일이 없습니다. 코드를 실행하기 전에 파일을 추가해주세요.")
else:
    # 모든 CSV 파일을 읽어 리스트에 담습니다. stock_code의 값이 변경 되는 것을 방지하기위해 명시적으로 stock_code는 문자열로 가져오라고 알려줍니다.
    print(f"중복 제거 할 총 파일 갯수 : {len(csv_files)}")
    for file in csv_files :
        df = pd.read_csv(file, dtype={'stock_code': str})
        print(f"중복 제거 전 {file}의 레코드 수 : {len(df)}")
        before_data_row += len(df)
        df = df.drop_duplicates(subset=["article_url"], keep="first")
        print(f"중복 제거 후 {file}의 레코드 수 : {len(df)}")
        after_data_row += len(df)
        df['stock_code'] = df['stock_code'].astype(str).str.zfill(6)
        for col in numeric_columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)
        df['article_comments'] = df['article_comments'].fillna('')
        df['article_comments'] = df['article_comments'].apply(clean_text)
        df['article_comments'] = df['article_comments'].apply(lambda x: [comment.strip() for comment in x.split('||')] if x else [])
        df['article_content'] = df['article_content'].apply(clean_text)
        df.reset_index(drop=True, inplace=True)
        df_list.append(df)
        output_file_base = file.replace(data_dir, cleaned_data_dir).replace(".csv", "")
        df.to_csv(output_file_base+'_cleaned.csv', index=False, encoding='utf-8-sig')
        df.to_json(output_file_base+'_cleaned.json', indent=4, force_ascii=False)
        print(f"전처리된 데이터가 csv, json 포멧으로 저장되었습니다.")
    df_all = pd.concat(df_list, ignore_index=True)
    df_all.to_csv(os.path.join(cleaned_data_dir, 'all_naver_stock_articles_cleaned.csv'), index=False, encoding='utf-8-sig')
    df_all.to_json(os.path.join(cleaned_data_dir, 'all_naver_stock_articles_cleaned.json'), indent=4, force_ascii=False)
    print("\n")
    print("-"*50)
    print(f"전처리된 모든 데이터가 {cleaned_data_dir} 폴더에 파일 csv, json로 저장되었습니다.")
    print(f"중복제거 전 전체 데이터 레코드 수 : {before_data_row}")
    print(f"중복제거 후 전체 데이터 레코드 수 : {after_data_row}")
    print(f"전체 데이터의 레코드 수 : {len(df_all)}")

중복 제거 할 총 파일 갯수 : 64
중복 제거 전 output/naver_stock_articles_20대_안철수_004770.csv의 레코드 수 : 720761
중복 제거 후 output/naver_stock_articles_20대_안철수_004770.csv의 레코드 수 : 5354
전처리된 데이터가 csv, json 포멧으로 저장되었습니다.
중복 제거 전 output/naver_stock_articles_20대_안철수_013700.csv의 레코드 수 : 131092
중복 제거 후 output/naver_stock_articles_20대_안철수_013700.csv의 레코드 수 : 2272
전처리된 데이터가 csv, json 포멧으로 저장되었습니다.
중복 제거 전 output/naver_stock_articles_20대_안철수_026910.csv의 레코드 수 : 186
중복 제거 후 output/naver_stock_articles_20대_안철수_026910.csv의 레코드 수 : 62
전처리된 데이터가 csv, json 포멧으로 저장되었습니다.
중복 제거 전 output/naver_stock_articles_20대_안철수_049480.csv의 레코드 수 : 14816
중복 제거 후 output/naver_stock_articles_20대_안철수_049480.csv의 레코드 수 : 756
전처리된 데이터가 csv, json 포멧으로 저장되었습니다.
중복 제거 전 output/naver_stock_articles_20대_안철수_053800.csv의 레코드 수 : 12090096
중복 제거 후 output/naver_stock_articles_20대_안철수_053800.csv의 레코드 수 : 21977
전처리된 데이터가 csv, json 포멧으로 저장되었습니다.
중복 제거 전 output/naver_stock_articles_20대_윤석열_004830.csv의 레코드 수 : 1819006
중복 제거 후 output/naver_stock_articles_20대_윤